### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

### Ingesting Data

In [412]:
train = tf.io.read_file('datasets/train.csv')
test =  tf.data.Dataset.list_file('datasets/train.csv')

train

AttributeError: type object 'DatasetV2' has no attribute 'list_file'

In [ ]:
test.head()

### Data Exploration

In [ ]:
train.info()

In [ ]:
train.describe()

### Variable Distribution

In [ ]:
train.columns

In [ ]:
# grid = sns.FacetGrid(train)
# grid.map_dataframe(sns.histplot)
# plt.savefig('histograms.jpg')

## Missing Data

In [ ]:
def check_missing(df=train):
    missing = df.isna().sum()
    missing_data = missing[missing > 0]
    return missing_data

check_missing()

In [ ]:
msno.matrix(train)

In [ ]:
msno.heatmap(train)

# Fill Missing Data

### YearBuilt

In [ ]:
corr = train[["GarageYrBlt", "YearBuilt"]].corr()
corr

In [ ]:
train["GarageYrBlt"].fillna(train["YearBuilt"],inplace=True)

## LotFrontage

In [ ]:
check_missing()

In [ ]:
sns.catplot(data=train, x='LotFrontage',kind='strip', row='LotConfig', sharex=False,height=3,orient='portrait')

plt.show()


In [ ]:
lot_front_dic = train.mask(train.LotFrontage.isna()) \
            .groupby('LotConfig') \
                ['LotFrontage'].median().round() \
                .to_dict()

lot_front_dic

In [ ]:
missing_LF = train[train.LotFrontage.isna()].index  

for key, item in lot_front_dic.items():
    train.loc[train['LotConfig']== key, 'LotFrontage'].fillna(item, inplace=True)

In [ ]:
check_missing()

# Drop Columns

### Drop 8 rows of ManVnrArea

In [ ]:
train.dropna(axis = 'rows', subset=["MasVnrArea"], inplace=True)

### Drop categorical columns  with more than 600 missing values

In [ ]:
t = int(train.shape[0]*.90)
train.dropna(axis='columns', thresh= t, inplace=True)

### Drop Columns Dominated by a certain value

In [ ]:
unbalanced_cols = []
for col in train.select_dtypes('object').columns:
    value_counts = train[col].value_counts(normalize=True)

    max = value_counts.max()
    
    print(value_counts)
   # print(max)

    if max > .70:
        unbalanced_cols.append(col)


In [ ]:
train.drop(unbalanced_cols,axis='columns',inplace=True)
check_missing()

In [ ]:
train.dropna(axis=0,inplace=True)
print(train.shape)
check_missing()

In [ ]:
y_train = train.pop('SalePrice')
x_train = train
check_missing(train_final)

## Load TensorBoard

In [404]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [405]:
import tensorflow as tf
import datetime, os

In [406]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4422102/4422102 [==============================] - 8s 2us/step


In [407]:
def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28), name='layers_flatten'),
    tf.keras.layers.Dense(512, activation='relu', name='layers_dense'),
    tf.keras.layers.Dropout(0.2, name='layers_dropout'),
    tf.keras.layers.Dense(10, activation='softmax', name='layers_dense_2')
  ])

In [408]:
def train_model():

  model = create_model()
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(x=x_train, 
            y=y_train, 
            epochs=5, 
            validation_data=(x_test, y_test), 
            callbacks=[tensorboard_callback])

train_model()

Epoch 1/5
1875/1875 [==============================] - 35s 17ms/step - loss: 0.4975 - accuracy: 0.8226 - val_loss: 0.4266 - val_accuracy: 0.8441
Epoch 2/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.3805 - accuracy: 0.8612 - val_loss: 0.3757 - val_accuracy: 0.8606
Epoch 3/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.3487 - accuracy: 0.8719 - val_loss: 0.3597 - val_accuracy: 0.8681
Epoch 4/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.3303 - accuracy: 0.8782 - val_loss: 0.3425 - val_accuracy: 0.8764
Epoch 5/5
1875/1875 [==============================] - 28s 15ms/step - loss: 0.3145 - accuracy: 0.8834 - val_loss: 0.4075 - val_accuracy: 0.8540
